# 01_import dataset

## 01-1_import data

In [1]:
import os
import json
import pandas as pd

#https://drive.google.com/drive/folders/18qV82fNY3IIWu3BRoGqm_LNgJzE8Akbr?usp=drive_link
#base_dir = "/Users/Andypon/10_交大研究所/1141_01_機器學習與金融科技/data"
base_dir= '/Users/andyw.p.chen/Documents/Project/datasets'
#base_dir=  "c:\Users\user\Downloads\datasets"

def load_json_to_df(filename: str) -> pd.DataFrame:
    file_path = os.path.join(base_dir, filename)
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # 如果是 { "target": {id: value, ...} }
    if isinstance(data, dict) and len(data) == 1 and isinstance(next(iter(data.values())), dict):
        key, inner = next(iter(data.items()))
        return pd.DataFrame(list(inner.items()), columns=["id", key])

    # dict of scalar
    if isinstance(data, dict):
        return pd.DataFrame([{"code": k, "desc": v} for k, v in data.items()])

    # list of dict
    elif isinstance(data, list):
        return pd.DataFrame(data)

    else:
        raise ValueError(f"Unsupported JSON structure in {filename}: {type(data)}")


def load_csv_to_df(filename: str) -> pd.DataFrame:
    """讀取 CSV 並轉為 DataFrame。"""
    return pd.read_csv(os.path.join(base_dir, filename))

# JSON 資料
##mcc_codes_df = load_json_to_df("mcc_codes.json")
train_fraud_labels_df = load_json_to_df("train_fraud_labels.json")

# CSV 資料
cards_df = load_csv_to_df("cards_data.csv")
transactions_df = load_csv_to_df("transactions_data.csv")
users_df = load_csv_to_df("users_data.csv")



## 01-2_rename variable in each data set

In [2]:
train_fraud_labels_df = train_fraud_labels_df.rename(columns={'id': 'transactions_id'})
train_fraud_labels_df = train_fraud_labels_df.rename(columns={'target': 'is_fraud'})

cards_df = cards_df.rename(columns={'id':'card_id'})

users_df = users_df.rename(columns={'id':'client_id'})

transactions_df = transactions_df.rename(columns={'mcc': 'mcc_code'})
transactions_df = transactions_df.rename(columns={'id': 'transaction_id'})

## 01-3_變數型態統一及缺失值處理

In [3]:
def add_missing_flags(df: pd.DataFrame, cols: list) -> pd.DataFrame:
    """
    在 DataFrame 中對指定欄位建立 missing flag 欄位
    flag=1 表示缺失值，flag=0 表示非缺失值
    
    參數
    ----
    df : pd.DataFrame
        輸入的資料框
    cols : list
        要檢查的欄位名稱清單
    
    回傳
    ----
    pd.DataFrame : 新的資料框 (含新增的 flag 欄位)
    """
    for col in cols:
        df[f"{col}_missing_flag"] = df[col].isna().astype(int)
    return df

transactions_df = add_missing_flags(transactions_df, ["merchant_state", "zip", "errors"])

##train_fraud_labels_df##
train_fraud_labels_df["is_fraud"]=train_fraud_labels_df["is_fraud"].astype("category") 
train_fraud_labels_df["transactions_id"]=train_fraud_labels_df["transactions_id"].astype(int) #合併資料需要

##cards_df##
cards_df["card_brand"]=cards_df["card_brand"].astype("category") 
cards_df["card_type"]=cards_df["card_type"].astype("category")
#####不要load這行 cards_df["expires"]=pd.to_datetime(cards_df["expires"], format="%m/%Y")
cards_df["expires"] = pd.to_datetime(cards_df["expires"], format="%m/%Y").dt.to_period("M")
cards_df["has_chip"]=cards_df["has_chip"].astype("category")

cards_df['credit_limit'] = cards_df['credit_limit'].replace(r'[\$,]', '', regex=True).astype(int)
#####不要load這行 cards_df["acct_open_date"]=pd.to_datetime(cards_df["acct_open_date"], format="%m/%Y")
cards_df["acct_open_date"] = pd.to_datetime(cards_df["acct_open_date"], format="%m/%Y").dt.to_period("M")
#####不要load這行 cards_df["year_pin_last_changed"]=pd.to_datetime(cards_df["year_pin_last_changed"], format="%Y")
cards_df["year_pin_last_changed"] = pd.to_datetime(cards_df["year_pin_last_changed"], format="%Y").dt.to_period("Y")
cards_df["card_on_dark_web"]=cards_df["card_on_dark_web"].astype("category") 

##users_df##
users_df["birth_year"] = pd.to_datetime(users_df["birth_year"], format="%Y").dt.to_period("Y")
users_df["birth_month"] = pd.to_datetime(users_df["birth_month"], format="%m").dt.to_period("M")
users_df["gender"]=users_df["gender"].astype("category") 
users_df['per_capita_income'] = users_df['per_capita_income'].replace(r'[\$,]', '', regex=True).astype(int)
users_df['yearly_income'] = users_df['yearly_income'].replace(r'[\$,]', '', regex=True).astype(int)
users_df['total_debt'] = users_df['total_debt'].replace(r'[\$,]', '', regex=True).astype(int)

##transactions_df##
transactions_df["date"] = pd.to_datetime(transactions_df["date"])
#浮點數轉整數原因確定？
transactions_df['amount'] = transactions_df['amount'].replace(r'[\$,]', '', regex=True).astype(float).astype(int)
##負數取log調成1
#transactions_df['amount'] = transactions_df['amount'].replace(r'[\$,]', '', regex=True).astype(float)

transactions_df["use_chip"]=transactions_df["use_chip"].astype("category") 

transactions_df.loc[
    transactions_df['merchant_city'].str.lower() == 'online',
    'merchant_state'
] = 'online'

transactions_df.loc[
    transactions_df['merchant_city'].str.lower() == 'online',
    'zip'
] = 20000 #原本是-1
## 我沒有全部改，這樣完之後仍有89006筆Missing，剩下都是在國外
transactions_df['zip'] = transactions_df['zip'].fillna(10000) #原本是-999
transactions_df["zip"]=transactions_df["zip"].astype("int64")

transactions_df['errors'] = transactions_df['errors'].astype('category')
transactions_df['errors'] = transactions_df['errors'].cat.add_categories('No_error').fillna('No_error')

#cars one hot encoding
##統一類別變數轉dummy variable(要注意共線性問題，應刪掉其中之一)

#card_type 原始種類：Debit_57%, Credit_33%, Debit(Prepaid)_9%
#card_brand 原始種類：MasterCard_52%, Visa_38%, Amex_7%, Discovery_3%
#has_chip 原始種類：Yes_89%, No_11%
#card_on_dark_web 原始種類：No_0%
cols_to_encode = ['card_type', 'card_brand', 'has_chip']
cards_df[cols_to_encode] = cards_df[cols_to_encode].astype('category')
dummies_cards = pd.get_dummies(
    cards_df[cols_to_encode], 
    prefix=cols_to_encode, 
    dtype='uint8'
    )
cards_df = pd.concat([cards_df, dummies_cards], axis=1)

#use_chip 原始種類：Swiped_52%, Chipe_36%, Online_12%
dummies_use = pd.get_dummies(transactions_df['use_chip'], prefix='use_chip', dtype='uint8')
transactions_df = pd.concat([transactions_df, dummies_use], axis=1)

#gender 原始種類：Female_51%, Male_49%
dummies_gender = pd.get_dummies(users_df['gender'], prefix='gender', dtype='uint8')
users_df = pd.concat([users_df, dummies_gender], axis=1)


cards_df.drop(columns=["has_chip_NO","has_chip"], inplace=True)
transactions_df.drop(columns=["use_chip"], inplace=True)
users_df.drop(columns=["gender_Female"], inplace=True)

## 01-4_資料整併

In [4]:
#transactions_df.loc[transactions_df["transaction_id"] == 10649266] #transaction_id vs id

#原始資料筆數：13305915
### transactions_df+train_fraud_labels_df      left 會有4390952 missing values
merged = pd.merge(transactions_df, train_fraud_labels_df, left_on="transaction_id", right_on="transactions_id", how="outer")
### transactions_df train_fraud_labels_df(8914963) + users_df 對過去不會有missing values
merged = pd.merge(merged,users_df , left_on="client_id", right_on="client_id", how="left")
### transactions_df train_fraud_labels_df users_df + cards_df 對過去不會有missing values
merged = pd.merge(merged,cards_df , left_on="card_id", right_on="card_id", how="left")

#刪掉重複的columns
merged.drop(columns=["transactions_id"], inplace=True)
merged.drop(columns=["client_id_y"], inplace=True)

## 合併完之後最後處理is_fraud(原會有missing values問題)
merged["is_fraud"] = merged["is_fraud"].astype(str)
merged.loc[merged['is_fraud'].str.lower() == 'no','is_fraud'] = '0'
merged.loc[merged['is_fraud'].str.lower() == 'yes','is_fraud'] = '1'
merged["is_fraud"] = pd.to_numeric(merged["is_fraud"], errors="coerce").astype("Int64")

merged = add_missing_flags(merged, ["is_fraud"])

#merged.to_csv("merged.csv", index=False)

# 先刪除不需要的DataFrame以節省記憶體
del transactions_df, users_df, cards_df, train_fraud_labels_df, cols_to_encode, dummies_cards, dummies_use, dummies_gender

In [5]:
backup_merged = merged.copy()
#merged = backup_merged.copy()

# 02_Research Design
## 02-1_RFM model/ domain knowledge features engineering model

In [8]:
##有出事再趕快回復原狀
merged = backup_merged.copy()

In [9]:
import numpy as np
import pandas as pd

# 確保日期是 datetime 並排序
merged['date'] = pd.to_datetime(merged['date'])
merged = merged.sort_values(by=['client_id_x', 'date']).reset_index(drop=True)

# --- RecencyInterval ---
merged['RecencyInterval'] = merged.groupby('client_id_x')['date'].diff().dt.total_seconds().fillna(0)/60

# --- TxnFrequency for multiple windows (向量化滑動窗口) ---
window_days = [7, 30, 60, 90]
for w in window_days:
    merged[f'TxnFrequency_{w}d'] = 0

def compute_freq_vectorized(dates, windows):
    """向量化計算每筆交易在每個 window 內的交易數"""
    n = len(dates)
    dates_int = dates.values.astype('datetime64[D]').astype(int)
    res = {w: np.zeros(n, dtype=int) for w in windows}
    for w in windows:
        left = 0
        counts = np.zeros(n, dtype=int)
        for right in range(n):
            while dates_int[right] - dates_int[left] > w:
                left += 1
            counts[right] = right - left + 1
        res[w] = counts
    return res

# 分組計算
for cid, g in merged.groupby('client_id_x', sort=False):
    freq_dict = compute_freq_vectorized(g['date'], window_days)
    for w in window_days:
        merged.loc[g.index, f'TxnFrequency_{w}d'] = freq_dict[w]

# --- AmtDelta ---
merged['prev_amount'] = merged.groupby('client_id_x')['amount'].shift(1)
merged['AmtDelta'] = merged['amount'] - merged['prev_amount']
merged['AmtDelta'] = merged['AmtDelta'].fillna(0)
merged.drop(columns='prev_amount', inplace=True)

In [10]:
# US region mapping
us_region_map = {
    'Northeast': ['NY','NJ','PA','MA','CT','RI','NH','VT','ME'],
    'Midwest': ['IL','OH','MI','IN','WI','MN','IA','MO','ND','SD','NE','KS'],
    'South': ['FL','GA','SC','NC','AL','MS','LA','TX','OK','TN','KY','VA','WV','AR','MD','DE','DC'],
    'West': ['CA','WA','OR','NV','AZ','NM','CO','UT','ID','MT','WY','AK','HI'],
}
continent_map = {
    'Europe': [ ... ],  # 原本 continent_map['Europe'] 可直接使用
    'Online': ['online','AA']
}

us_region_lookup = {state: region for region, states in us_region_map.items() for state in states}

# --- 向量化 location 特徵 ---
merged['merchant_online'] = merged['merchant_state'].eq('online').astype('uint8')
merged['merchant_us'] = merged['merchant_state'].isin(us_region_lookup.keys()).astype('uint8')
merged['merchant_eu'] = merged['merchant_state'].isin(continent_map['Europe']).astype('uint8')
merged['merchant_others'] = (~merged[['merchant_online','merchant_us','merchant_eu']].any(axis=1)).astype('uint8')


In [11]:
# --- 首次交易標記 ---
merged['FirstTxnInRegion'] = (~merged.duplicated(subset=['client_id_x', 'merchant_state'])).astype('uint8')

In [12]:
# （1）先依 client 排序（確保交易順序正確）
merged = merged.sort_values(['client_id_x', 'date'])

# （2）取得同一客戶上一筆交易的州
merged['prev_state'] = merged.groupby('client_id_x')['merchant_state'].shift(1)

# （3）當前 state ≠ 上一筆 state → 1，否則 0
merged['DifferentState'] = (merged['merchant_state'] != merged['prev_state']).astype('uint8')


## 02-2_data transformation

In [13]:
#merged[["card_id","card_number"]]
import numpy as np
from scipy import stats 

# === (1) log轉換 ===
merged['amount'] = np.where(merged['amount'] < 0, 0, merged['amount'])  # 負數變 0
merged['amount'] = np.log(merged['amount'] + 1)  

# === (3) 平方根轉換 ===
merged['credit_limit']=np.sqrt(merged['credit_limit'])
merged['total_debt']=np.sqrt(merged['total_debt'])

# === (3) 立方根轉換 ===
merged['yearly_income']=np.cbrt(merged['yearly_income'])
merged['per_capita_income']=np.cbrt(merged['per_capita_income'])

## 02-3_Spilt data into training/ test set

In [14]:
# --- 選取數值型變數 ---
num_cols = merged.select_dtypes(include=['int64', 'float64','uint8','datetime64[ns]']).columns
df2 = merged[num_cols]

# --- dropna ---
df_cleaned = df2.dropna()
del df2

# --- 避免共線性 ---
df_cleaned.drop(columns=["is_fraud_missing_flag","card_type_Debit (Prepaid)", 
                         "card_brand_Discover", "use_chip_Online Transaction"], inplace=True)

# --- 確保 date 欄位在 df_cleaned 中 ---
if 'date' not in df_cleaned.columns:
    df_cleaned['date'] = merged.loc[df_cleaned.index, 'date']

# --- 依時間排序 ---
df_sorted = df_cleaned.sort_values('date')

# --- 時間序列切分（前 80% 訓練, 後 20% 測試） ---
split_index = int(len(df_sorted) * 0.8)
train_df = df_sorted.iloc[:split_index].drop(columns=['date'])  # 可選擇丟掉 date
test_df  = df_sorted.iloc[split_index:].drop(columns=['date'])

# --- 檢查詐欺資料分布 ---
print(train_df['is_fraud'].value_counts(normalize=False))
print(test_df['is_fraud'].value_counts(normalize=False))

# --- 清理不用的變數 ---
del df_cleaned, df_sorted, merged

is_fraud
0    7121755
1      10215
Name: count, dtype: Int64
is_fraud
0    1779876
1       3117
Name: count, dtype: Int64


## 02-4_high fraud rate calculation

In [15]:
# -----------------------------
# 1. 用 train_df 計算 fraud rate
# -----------------------------
fraud_rate_train = (
    train_df.groupby('mcc_code')['is_fraud'].mean()
)

# 設定 threshold = 2%
high_risk_MCC_list = fraud_rate_train[fraud_rate_train > 0.02].index.tolist()

# -----------------------------
# 2. 加入特徵到 train_df / test_df
# -----------------------------
train_df['HighRiskMCC'] = train_df['mcc_code'].isin(high_risk_MCC_list).astype('uint8')
test_df['HighRiskMCC']  = test_df['mcc_code'].isin(high_risk_MCC_list).astype('uint8')

## 02-5_Avoid Multicollinearity

In [16]:
##處理共線性
train_df.drop(columns=["per_capita_income"], inplace=True)
train_df.drop(columns=["use_chip_Chip Transaction","merchant_state_missing_flag","zip_missing_flag"], inplace=True)           
train_df.drop(columns=["card_brand_Visa" ,"card_brand_Amex","card_type_Credit"], inplace=True)
test_df.drop(columns=["per_capita_income"], inplace=True)
test_df.drop(columns=["use_chip_Chip Transaction","merchant_state_missing_flag","zip_missing_flag"], inplace=True)           
test_df.drop(columns=["card_brand_Visa" ,"card_brand_Amex","card_type_Credit"], inplace=True)

# 03_Model 1-Stepwise feature selection

In [ ]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.metrics import roc_auc_score, average_precision_score, confusion_matrix


def stepwise_logit_with_k_tables(train_df, test_df, dep_var="is_fraud", k=314657018,
                                 threshold_in=0.05, threshold_out=0.10, verbose=True):
    """
    Stepwise logistic regression (forward + backward) with flexible k control,
    and 3 formatted output tables like table_for_first_step().
    """

    y_train = train_df[dep_var]
    X_train = train_df.drop(columns=[dep_var])
    y_test = test_df[dep_var]
    X_test = test_df.drop(columns=[dep_var])

    included = []
    step = 0
    full_mode = (k == 314657018)

    # ========= 🔸 特殊情況：k = 0，只跑 intercept =========
    if k == 0:
        final_model = sm.Logit(y_train, sm.add_constant(np.ones(len(y_train)))).fit(disp=False)
        ll_full = final_model.llf
        ll_null = ll_full

        overall_fit = pd.DataFrame({
            "Measure": ["-2 Log Likelihood (−2LL) value"],
            "Value": [round(-2 * ll_full, 3)],
            "Change_from_Base": [""],
            "Change_pvalue": [""]
        })

        coef_df = pd.DataFrame({
            "Independent Variable": ["const"],
            "B": final_model.params.values,
            "Std. Error": final_model.bse.values,
            "Wald": [np.nan],
            "df": [1],
            "Sig.": [""],
            "Exp(B)": np.exp(final_model.params.values)
        })

        not_in_eq_df = pd.DataFrame({
            "Independent Variable": X_train.columns,
            "Score Statistic (LRT)": [None]*len(X_train.columns),
            "Significance": [None]*len(X_train.columns)
        })

        print(f"\n✅ Stepwise completed with 0 variables (Intercept only).")
        return overall_fit, coef_df, not_in_eq_df, final_model
    # ======================================================

    # ---------- Regular Stepwise ----------
    while True:
        step += 1
        changed = False

        # ---------- Forward Step ----------
        excluded = list(set(X_train.columns) - set(included))
        new_pvals = pd.Series(index=excluded, dtype=float)
        for new_var in excluded:
            try:
                model = sm.Logit(y_train, sm.add_constant(X_train[included + [new_var]])).fit(disp=False)
                new_pvals[new_var] = model.pvalues[new_var]
            except Exception:
                new_pvals[new_var] = np.nan

        if new_pvals.empty:
            break

        best_pval = new_pvals.min()
        if best_pval < threshold_in:
            best_var = new_pvals.idxmin()
            included.append(best_var)
            changed = True
            if verbose:
                print(f"🟢 Step {step}: Forward — added variable: {best_var} (p={best_pval:.4g})")

        # ---------- Backward Step ----------
        if included:
            model = sm.Logit(y_train, sm.add_constant(X_train[included])).fit(disp=False)
            pvalues = model.pvalues.iloc[1:]  # skip intercept
            worst_pval = pvalues.max()
            if worst_pval > threshold_out:
                worst_var = pvalues.idxmax()
                included.remove(worst_var)
                changed = True
                if verbose:
                    print(f"🔴 Step {step}: Backward — removed variable: {worst_var} (p={worst_pval:.4g})")

        # ---------- 結束條件 ----------
        if not changed:
            if verbose:
                print(f"⚪ Step {step}: No change — stopping iteration.")
            break
        if not full_mode and len(included) >= k:
            if verbose:
                print(f"🟡 Reached k={k}, stopping after {len(included)} variables.")
            break

    # ========= Final Model =========
    final_model = sm.Logit(y_train, sm.add_constant(X_train[included])).fit(disp=False)
    ll_full = final_model.llf
    ll_null = sm.Logit(y_train, sm.add_constant(np.ones(len(y_train)))).fit(disp=False).llf

    # 1️⃣ Overall Model Fit
    ll_diff = -2 * (ll_null - ll_full)
    df_diff = len(final_model.params) - 1
    p_value = stats.chi2.sf(ll_diff, df_diff)

    overall_fit = pd.DataFrame({
        "Measure": [
            "-2 Log Likelihood (−2LL) value",
            "Cox and Snell R2",
            "Nagelkerke R2",
            "Pseudo R2 (McFadden)",
            "Hosmer-Lemeshow χ2"
        ],
        "Value": [
            round(-2 * ll_full, 3),
            round(1 - np.exp((2 / len(y_train)) * (ll_null - ll_full)), 3),
            round((1 - np.exp((2 / len(y_train)) * (ll_null - ll_full))) / (1 - np.exp(2 * ll_null / len(y_train))), 3),
            round(1 - (ll_full / ll_null), 3),
            round(ll_diff, 3)
        ],
        "Change_from_Base": [
            round(-2 * (ll_null - ll_full), 3),
            "", "", "", ""
        ],
        "Change_pvalue": [
            round(p_value, 4),
            "", "", "", ""
        ]
    })

    # 2️⃣ Variables in the Equation
    coef_df = pd.DataFrame({
        "Independent Variable": final_model.params.index,
        "B": final_model.params.values,
        "Std. Error": final_model.bse.values,
        "Wald": (final_model.params / final_model.bse) ** 2,
        "df": 1,
        "Sig.": final_model.pvalues.values,
        "Exp(B)": np.exp(final_model.params.values)
    }).reset_index(drop=True)

    # 3️⃣ Variables Not in Equation
    excluded_vars = [v for v in X_train.columns if v not in included]
    not_in_eq = []
    for var in excluded_vars:
        try:
            temp_model = sm.Logit(y_train, sm.add_constant(X_train[included + [var]])).fit(disp=False)
            lr_stat = -2 * (final_model.llf - temp_model.llf)
            p_val = stats.chi2.sf(lr_stat, 1)
            not_in_eq.append({
                "Independent Variable": var,
                "Score Statistic (LRT)": round(lr_stat, 3),
                "Significance": round(p_val, 4)
            })
        except Exception:
            not_in_eq.append({
                "Independent Variable": var,
                "Score Statistic (LRT)": None,
                "Significance": None
            })

    not_in_eq_df = pd.DataFrame(not_in_eq)

    # ========= Train / Test Accuracy =========
    train_pred = (final_model.predict(sm.add_constant(X_train[included])) > 0.5).astype(int)
    test_pred = (final_model.predict(sm.add_constant(X_test[included])) > 0.5).astype(int)
    train_acc = (train_pred == y_train).mean()
    test_acc = (test_pred == y_test).mean()

    print(f"\n✅ Stepwise completed with {len(included)} variables: {included}")
    print(f"Train Accuracy:   {train_acc:.4f}")
    print(f"Test Accuracy:    {test_acc:.4f}")

    # ========= AUC & PR-AUC =========
    train_probs = final_model.predict(sm.add_constant(X_train[included]))
    test_probs = final_model.predict(sm.add_constant(X_test[included]))

    train_auc = roc_auc_score(y_train, train_probs)
    test_auc = roc_auc_score(y_test, test_probs)
    train_pr_auc = average_precision_score(y_train, train_probs)
    test_pr_auc = average_precision_score(y_test, test_probs)

    print(f"Train AUC:        {train_auc}")
    print(f"Test AUC:         {test_auc}")
    print(f"Train PR-AUC:     {train_pr_auc}")
    print(f"Test PR-AUC:      {test_pr_auc}")

    return overall_fit, coef_df, not_in_eq_df, final_model



# -----------------------------------------------------------
# 混淆矩陣分類表
# -----------------------------------------------------------
def classification_table(model, df, target_col="is_fraud", cutoff=0.0015):
    y_true = df[target_col].astype(int)
    X = sm.add_constant(df[model.params.index.drop("const")])
    y_pred_prob = model.predict(X)
    y_pred = (y_pred_prob >= cutoff).astype(int)

    cm = confusion_matrix(y_true, y_pred, labels=[1, 0])
    TP, FN, FP, TN = cm.ravel()

    fraud_total = TP + FN
    normal_total = FP + TN

    fraud_correct = TP / fraud_total if fraud_total > 0 else 0
    normal_correct = TN / normal_total if normal_total > 0 else 0
    overall_correct = (TP + TN) / (fraud_total + normal_total)

    table = pd.DataFrame({
        "Actual Group": ["Fraud (1)", "Normal (0)", "Total"],
        "Predicted Fraud (1)": [TP, FP, TP + FP],
        "Predicted Normal (0)": [FN, TN, FN + TN],
        "Total": [fraud_total, normal_total, fraud_total + normal_total],
        "% Correct": [
            round(fraud_correct * 100, 1),
            round(normal_correct * 100, 1),
            round(overall_correct * 100, 1),
        ],
        "cutoff": [cutoff, "", ""],
        "F1 Score": [round(2 * TP / (2 * TP + FP + FN), 4), "", ""],
    })
    return table



# ===========================================================
# 版本 2.0 (Full Model)
# ===========================================================
overall_fit, coef_df, not_in_eq_df, final_model = stepwise_logit_with_k_tables(
    train_df, test_df, dep_var="is_fraud", k=314657018, verbose=True
)

print("=== Overall Model Fit ===")
print(overall_fit)

print("\n=== Variables in the Equation ===")
print(coef_df)

print("\n=== Variables Not in the Equation ===")
print(not_in_eq_df)

print("\n================================================")
print("\n=== Accuracy in Training and Testing dataset ===")

train_table = classification_table(final_model, train_df, target_col="is_fraud")
test_table = classification_table(final_model, test_df, target_col="is_fraud")

print("\n=== Classification Matrix — Training Sample ===")
print(train_table.to_string(index=False))

print("\n=== Classification Matrix — Holdout (Test) Sample ===")
print(test_table.to_string(index=False))

# 03_Model 2-Lasso feature selection

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_recall_curve, average_precision_score

def fit_full_logit(train_df, test_df, dep_var="is_fraud"):
    # 1. split
    X_train = train_df.drop(columns=[dep_var])
    y_train = train_df[dep_var]

    X_test = test_df.drop(columns=[dep_var])
    y_test = test_df[dep_var]

    # 🎯 確保 test 欄位順序 = train 欄位順序
    X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

    # 2. 標準化
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 3. Logistic Regression
    model = LogisticRegression(
        penalty="l2",
        solver="saga",
        max_iter=200,
        n_jobs=-1
    )
    model.fit(X_train_scaled, y_train)

    # 4. probabilities
    train_pred = model.predict_proba(X_train_scaled)[:, 1]
    test_pred = model.predict_proba(X_test_scaled)[:, 1]

    # 5. ROC AUC
    train_auc = roc_auc_score(y_train, train_pred)
    test_auc = roc_auc_score(y_test, test_pred)

    # 6. PR AUC
    train_prauc = average_precision_score(y_train, train_pred)
    test_prauc = average_precision_score(y_test, test_pred)

    return {
        "model": model,
        "scaler": scaler,
        "train_pred": train_pred,
        "test_pred": test_pred,
        "train_auc": train_auc,
        "test_auc": test_auc,
        "train_prauc": train_prauc,
        "test_prauc": test_prauc
    }


In [ ]:
dep_var = "is_fraud"

# ⬇ 確保模型只吃 selected features + label
train_selected = train_df[selected_features + [dep_var]].copy()
test_selected = test_df[selected_features + [dep_var]].copy()

In [ ]:
result = fit_full_logit(train_selected, test_selected, dep_var="is_fraud")

print("Train AUC:", result["train_auc"])
print("Test AUC:", result["test_auc"])

print("Train PR-AUC:", result["train_prauc"])
print("Test PR-AUC:", result["test_prauc"])

# 03_Model 3-Ridge feature selection

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
import numpy as np

def lasso_select_features(train_df, dep_var="is_fraud"):

    y = train_df[dep_var]
    X = train_df.drop(columns=[dep_var])

    numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    X = X[numeric_cols]

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    lasso_cv = LogisticRegressionCV(
        Cs=np.logspace(-3, 2, 50),  # C from 0.001 to 100
        cv=5,
        penalty="l2",
        solver="saga",
        scoring="roc_auc",
        max_iter=10000,
        n_jobs=-1
    )

    lasso_cv.fit(X_scaled, y)

    coef = lasso_cv.coef_.ravel()
    selected = [col for col, c in zip(numeric_cols, coef) if abs(c) > 1e-6]

    best_C = lasso_cv.C_[0]
    best_lambda = 1 / best_C

    print("Best C:", best_C)
    print("Best lambda = 1/C:", best_lambda)

    print(f"Lasso 選到 {len(selected)} 個變數：")
    print(selected)

    return selected


selected_features = lasso_select_features(train_df, dep_var="is_fraud")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_recall_curve, average_precision_score

def fit_full_logit(train_df, test_df, dep_var="is_fraud"):
    # 1. split
    X_train = train_df.drop(columns=[dep_var])
    y_train = train_df[dep_var]

    X_test = test_df.drop(columns=[dep_var])
    y_test = test_df[dep_var]

    # 🎯 確保 test 欄位順序 = train 欄位順序
    X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

    # 2. 標準化
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 3. Logistic Regression
    model = LogisticRegression(
        penalty="l2",
        solver="saga",
        max_iter=200,
        n_jobs=-1
    )
    model.fit(X_train_scaled, y_train)

    # 4. probabilities
    train_pred = model.predict_proba(X_train_scaled)[:, 1]
    test_pred = model.predict_proba(X_test_scaled)[:, 1]

    # 5. ROC AUC
    train_auc = roc_auc_score(y_train, train_pred)
    test_auc = roc_auc_score(y_test, test_pred)

    # 6. PR AUC
    train_prauc = average_precision_score(y_train, train_pred)
    test_prauc = average_precision_score(y_test, test_pred)

    return {
        "model": model,
        "scaler": scaler,
        "train_pred": train_pred,
        "test_pred": test_pred,
        "train_auc": train_auc,
        "test_auc": test_auc,
        "train_prauc": train_prauc,
        "test_prauc": test_prauc
    }


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_recall_curve, average_precision_score

def fit_full_logit(train_df, test_df, dep_var="is_fraud"):
    # 1. split
    X_train = train_df.drop(columns=[dep_var])
    y_train = train_df[dep_var]

    X_test = test_df.drop(columns=[dep_var])
    y_test = test_df[dep_var]

    # 🎯 確保 test 欄位順序 = train 欄位順序
    X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

    # 2. 標準化
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 3. Logistic Regression
    model = LogisticRegression(
        penalty="l2",
        solver="saga",
        max_iter=200,
        n_jobs=-1
    )
    model.fit(X_train_scaled, y_train)

    # 4. probabilities
    train_pred = model.predict_proba(X_train_scaled)[:, 1]
    test_pred = model.predict_proba(X_test_scaled)[:, 1]

    # 5. ROC AUC
    train_auc = roc_auc_score(y_train, train_pred)
    test_auc = roc_auc_score(y_test, test_pred)

    # 6. PR AUC
    train_prauc = average_precision_score(y_train, train_pred)
    test_prauc = average_precision_score(y_test, test_pred)

    return {
        "model": model,
        "scaler": scaler,
        "train_pred": train_pred,
        "test_pred": test_pred,
        "train_auc": train_auc,
        "test_auc": test_auc,
        "train_prauc": train_prauc,
        "test_prauc": test_prauc
    }


In [ ]:
result = fit_full_logit(train_selected, test_selected, dep_var="is_fraud")

print("Train AUC:", result["train_auc"])
print("Test AUC:", result["test_auc"])

print("Train PR-AUC:", result["train_prauc"])
print("Test PR-AUC:", result["test_prauc"])

# 03_Model 4-Elastic Net feature selection

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

def elasticnet_select_features(train_df, dep_var="is_fraud"):
    """
    Elastic Net Logistic Regression (L1 + L2) 做變數選擇
    自動搜尋多個 l1_ratio
    """
    y = train_df[dep_var]
    X = train_df.drop(columns=[dep_var])

    numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    X = X[numeric_cols]

    # scale
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 更強的 grid + 多個 l1_ratio
    enet_cv = LogisticRegressionCV(
        Cs=np.logspace(-3, 2, 50),
        cv=5,
        penalty="elasticnet",
        solver="saga",
        scoring="roc_auc",
        max_iter=10000,
        class_weight="balanced",      # ★極度重要
        l1_ratios=[0.1, 0.3, 0.5, 0.7, 0.9],
        n_jobs=-1
    )

    enet_cv.fit(X_scaled, y)

    # 係數
    coef = enet_cv.coef_.ravel()
    selected = [col for col, c in zip(numeric_cols, coef) if abs(c) > 1e-6]

    print("Best C:", enet_cv.C_[0])
    print("Best l1_ratio:", enet_cv.l1_ratio_[0])
    print(f"Elastic Net 選出 {len(selected)} 個變數：")
    print(selected)

    return selected

selected_features = elasticnet_select_features(train_df, dep_var="is_fraud")


In [ ]:
dep_var = "is_fraud"

# ⬇ 確保模型只吃 selected features + label
train_selected = train_df[selected_features + [dep_var]].copy()
test_selected = test_df[selected_features + [dep_var]].copy()
result = fit_full_logit(train_selected, test_selected, dep_var="is_fraud")

print("Train AUC:", result["train_auc"])
print("Test AUC:", result["test_auc"])

print("Train PR-AUC:", result["train_prauc"])
print("Test PR-AUC:", result["test_prauc"])

# 03_Model 5- $X_{raw}$+ $X_{RFM}$+ $X_{DK}$+Elastic Net feature selection

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

def elasticnet_select_features(train_df, dep_var="is_fraud"):
    """
    Elastic Net Logistic Regression (L1 + L2) 做變數選擇
    自動搜尋多個 l1_ratio
    """
    y = train_df[dep_var]
    X = train_df.drop(columns=[dep_var])

    numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    X = X[numeric_cols]

    # scale
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 更強的 grid + 多個 l1_ratio
    enet_cv = LogisticRegressionCV(
        Cs=np.logspace(-3, 2, 50),
        cv=5,
        penalty="elasticnet",
        solver="saga",
        scoring="roc_auc",
        max_iter=10000,
        class_weight="balanced",      # ★極度重要
        l1_ratios=[0.1, 0.3, 0.5, 0.7, 0.9],
        n_jobs=-1
    )

    enet_cv.fit(X_scaled, y)

    # 係數
    coef = enet_cv.coef_.ravel()
    selected = [col for col, c in zip(numeric_cols, coef) if abs(c) > 1e-6]

    print("Best C:", enet_cv.C_[0])
    print("Best l1_ratio:", enet_cv.l1_ratio_[0])
    print(f"Elastic Net 選出 {len(selected)} 個變數：")
    print(selected)

    return selected

selected_features = elasticnet_select_features(train_df, dep_var="is_fraud")


In [ ]:
dep_var = "is_fraud"

# ⬇ 確保模型只吃 selected features + label
train_selected = train_df[selected_features + [dep_var]].copy()
test_selected = test_df[selected_features + [dep_var]].copy()
result = fit_full_logit(train_selected, test_selected, dep_var="is_fraud")

print("Train AUC:", result["train_auc"])
print("Test AUC:", result["test_auc"])

print("Train PR-AUC:", result["train_prauc"])
print("Test PR-AUC:", result["test_prauc"])

# 03_Model 6- $X_{raw}$+ $X_{RFM}$+ $X_{DK}$-identifier feature selection

In [ ]:
cols_to_drop = [
    'transaction_id',
    'client_id_x',
    'card_id',
    'card_number',
    'cvv'
]

train_df = train_df.drop(columns=cols_to_drop, errors='ignore')
test_df  = test_df.drop(columns=cols_to_drop, errors='ignore')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_recall_curve, average_precision_score

def fit_full_logit(train_df, test_df, dep_var="is_fraud"):
    # 1. split
    X_train = train_df.drop(columns=[dep_var])
    y_train = train_df[dep_var]

    X_test = test_df.drop(columns=[dep_var])
    y_test = test_df[dep_var]

    # 🎯 確保 test 欄位順序 = train 欄位順序
    X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

    # 2. 標準化
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 3. Logistic Regression
    model = LogisticRegression(
        penalty="l2",
        solver="lbfgs",
        max_iter=1000,
        n_jobs=-1
    )
    model.fit(X_train_scaled, y_train)

    # 4. probabilities
    train_pred = model.predict_proba(X_train_scaled)[:, 1]
    test_pred = model.predict_proba(X_test_scaled)[:, 1]

    # 5. ROC AUC
    train_auc = roc_auc_score(y_train, train_pred)
    test_auc = roc_auc_score(y_test, test_pred)

    # 6. PR AUC
    train_prauc = average_precision_score(y_train, train_pred)
    test_prauc = average_precision_score(y_test, test_pred)

    return {
        "model": model,
        "scaler": scaler,
        "train_pred": train_pred,
        "test_pred": test_pred,
        "train_auc": train_auc,
        "test_auc": test_auc,
        "train_prauc": train_prauc,
        "test_prauc": test_prauc
    }


In [ ]:
result = fit_full_logit(train_df, test_df, dep_var="is_fraud")

print("Train AUC:", result["train_auc"])
print("Test AUC:", result["test_auc"])

print("Train PR-AUC:", result["train_prauc"])
print("Test PR-AUC:", result["test_prauc"])

# 03_Model 7-10 
7. $X_{ALL}$ with XG Boost
8. $X_{raw}$+ $X_{RFM}$+ $X_{DK}$ with XG Boost
9. $X_{ALL}$ with LGBM
10. $X_{raw}$+ $X_{RFM}$+ $X_{DK}$ with LGBM

## 03_7-0 features grouping

In [26]:
import pandas as pd

result = pd.DataFrame(columns=[
    "Model", "Features", 
    "Train AUC", "Test AUC", 
    "Train PR AUC", "Test PR AUC"
])

# Raw features

raw_cols = [
    'amount', 'current_age',
    'retirement_age', 'yearly_income', 'total_debt',
    'credit_score', 'num_credit_cards', 'has_chip_YES',
    'num_cards_issued', 'credit_limit'
]

# RFM features
rfm_cols = [
    'RecencyInterval', 'TxnFrequency_7d','TxnFrequency_30d',
    'TxnFrequency_60d', 'TxnFrequency_90d','AmtDelta'
]

# DK features
dk_cols = [
    'merchant_online', 'merchant_us', 'merchant_eu', 'merchant_others',
    'FirstTxnInRegion','HighRiskMCC','DifferentState'
]

# Grouping
feature_groups = {
    "X_raw": raw_cols,
    "X_rfm": rfm_cols,
    "X_dk": dk_cols,
    "X_raw + X_rfm": raw_cols + rfm_cols,
    "X_raw + X_dk": raw_cols + dk_cols,
    "X_rfm + X_dk": rfm_cols + dk_cols,
    "X_raw + X_rfm + X_dk": raw_cols + rfm_cols + dk_cols
}

### XGB funciton

In [29]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score
import pandas as pd
import numpy as np

def run_xgb(df, feature_cols, dep_var="fraud_label", group_name="X_raw"):
    print("=" * 80)
    print(f"Running XGBoost for group: {group_name}")
    print(f"Using {len(feature_cols)} features")
    print("=" * 80)

    # ====== 1. Train-test split ======
    X = df[feature_cols]
    y = df[dep_var]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # ====== 2. XGBoost Model ======
    model = XGBClassifier(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        eval_metric="logloss",
        random_state=42,
        tree_method="hist",
        n_jobs=-1
    )

    # ====== 3. Fit with eval set ======
    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        early_stopping_rounds=50,
        verbose=False
    )

    # ====== 4. Predictions ======
    pred_train = model.predict_proba(X_train)[:, 1]
    pred_test = model.predict_proba(X_test)[:, 1]

    # ====== 5. Evaluation ======
    train_roc = roc_auc_score(y_train, pred_train)
    test_roc = roc_auc_score(y_test, pred_test)

    train_pr = average_precision_score(y_train, pred_train)
    test_pr = average_precision_score(y_test, pred_test)

    # ====== 6. Output ======
    print(f"[{group_name}] Results:")
    print(f"Train ROC AUC: {train_roc:.6f}")
    print(f"Test  ROC AUC: {test_roc:.6f}")
    print(f"Train PR  AUC: {train_pr:.6f}")
    print(f"Test  PR  AUC: {test_pr:.6f}")
    print("\n")
    return {
        "group": group_name,
        "train_roc": train_roc,
        "test_roc": test_roc,
        "train_pr": train_pr,
        "test_pr": test_pr
    }


### Light GBM function

In [22]:
def run_lgbm(train_df, test_df, feature_list, label_col, feature_name):
    global result

    # --- Extract X, y ---
    X_train = train_df[feature_list]
    y_train = train_df[label_col]

    X_test  = test_df[feature_list]
    y_test  = test_df[label_col]

    # --- Class imbalance handling ---
    pos = y_train.sum()
    neg = len(y_train) - pos
    spw = neg / pos

    # --- Model ---
    lgb_model = lgb.LGBMClassifier(
        n_estimators=500,
        learning_rate=0.02,
        max_bin=128,
        scale_pos_weight=spw,
        min_split_gain=1.0,
        reg_alpha=0.1,
        reg_lambda=5,
        min_child_samples=20,
        max_depth=4,
        subsample=0.8,
        colsample_bytree=0.8,
    )

    lgb_model.fit(X_train, y_train)

    # --- Predict ---
    train_pred = lgb_model.predict_proba(X_train)[:, 1]
    test_pred  = lgb_model.predict_proba(X_test)[:, 1]

    # --- Metrics ---
    train_auc = roc_auc_score(y_train, train_pred)
    test_auc  = roc_auc_score(y_test, test_pred)

    train_pr = average_precision_score(y_train, train_pred)
    test_pr  = average_precision_score(y_test, test_pred)

    # --- Save results ---
    result.loc[len(result)] = [
        "LightGBM",
        feature_name,
        train_auc, test_auc,
        train_pr, test_pr
    ]


### Model 7-10 result

In [ ]:
for group_name, group_cols in feature_groups.items():
    run_xgb(int_transactions_df, group_cols, "is_fraud", group_name)


In [ ]:
for group_name, group_cols in feature_groups.items():
    run_lgbm(train_df, test_df, group_cols, "is_fraud", group_name)

result